In [1]:
!pip3 install tweepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
def age2tag(age):
  age = int(age[:2])
  if age < 20:
    tag = '<10代>'
  elif age < 40:
    tag = '<20～30代>'
  elif age < 70:
    tag = '<40～60代>'
  else:
    tag = ''
  return tag

def gender2tag(gender):
  if gender == '男':
    tag = '<男性>'
  elif gender == '女':
    tag = '<女性>'
  else:
    tag = ''
  return tag

In [ ]:
import tweepy
import random
import re


while True:
    api_key = ''
    api_secret_key = ''
    access_token = ''
    access_token_secret = ''

    auth = tweepy.OAuthHandler(api_key, api_secret_key)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True)

    # botのツイートを除外するため，一般的なクライアント名を列挙
    sources = ["TweetDeck", "Twitter Web Client", "Twitter for iPhone",
               "Twitter for iPad", "Twitter for Android", "Twitter for Android Tablets",
               "ついっぷる", "Janetter", "twicca", "Keitai Web", "Twitter for Mac"]
    

    # ひらがな一文字で検索し，スクリーンネームを取得
    words = list("あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやゆよらりるれろわをん")
    
    # screen_name（ユーザー名）を取得している
    screen_names = set()
    for s in api.search(q=random.choice(words), lang='ja', result_type='recent', count=100, tweet_mode='extended'):
        if s.source in sources:
            screen_names.add(s.author.screen_name)

    # ステータスidからステータスを得るためのdict
    id2status = {}

    # スクリーンネームからタイムラインを取得してツイートを保存．
    # さらにリプライツイートであれば，リプライ先のスクリーンネームも取得
    in_reply_to_screen_names = set()
    for name in screen_names:
        try:            
            for s in api.user_timeline(name, tweet_mode='extended', count=200):
                # リンクもしくはハッシュタグを含むツイートは除外する
                if "http" not in s.full_text and "#" not in s.full_text:
                    id2status[s.id] = s
                    if s.in_reply_to_screen_name is not None:
                        if s.in_reply_to_screen_name not in screen_names:
                            in_reply_to_screen_names.add(s.in_reply_to_screen_name)
        except Exception as e:
            continue

    # リプライ先のスクリーンネームからタイムラインを取得してツイートを保存
    # 例）年齢・性別を含むツイートを表示する
    pattern_age = re.compile('[1-6][0-9][歳代]')
    pattern_gender = re.compile("(男|女)")
    for name in in_reply_to_screen_names:
        try:
            user = api.get_user(screen_name=name)
            if re.search(pattern_age, user.description) or re.search(pattern_gender, user.description):
                for s in api.user_timeline(name, tweet_mode='extended', count=300):
                    if "http" not in s.full_text and "#" not in s.full_text:
                        id2status[s.id] = s
        except Exception as e:
            continue

    # 保存したツイートのリプライ先のツイートが保存されていれば，id2replyidのキーを元ツイートのid，値をリプライ先ツイートのidとする
    id2replyid = {}
    for _, s in id2status.items():
        if s.in_reply_to_status_id in id2status:
            #元ツイートとリプライのユーザーが同じなら省く
            if id2status[s.id].author != id2status[s.in_reply_to_status_id].author:
                id2replyid[s.in_reply_to_status_id] = s.id
            #確認用
            # else:
            #   print(id2status[s.id].author.screen_name, id2status[s.in_reply_to_status_id].author.screen_name)
            #   print(id2status[s.id].full_text, id2status[s.in_reply_to_status_id].full_text)

    # id2replyidのkey valueからstatusを取得し，ツイートペアをタブ区切りで保存
    # 例）年齢・性別を含むツイートを表示する
    pattern_age = re.compile('[1-6][0-9][歳代]')
    pattern_gender = re.compile("(男|女)")
    f = open("tweet_pairs.txt", "a")
    f1 = open("reply_user.txt", "a")
    for id, rid in id2replyid.items():
        user = api.get_user(screen_name=id2status[rid].author.screen_name)

        # 年齢タグの作成
        if re.search(pattern_age, user.description):
          tag_age = age2tag(''.join(re.findall(pattern_age, user.description)[0]))
        else:
          tag_age = ''

        # 性別タグの作成
        if re.search(pattern_gender, user.description):
          tag_gender = gender2tag(''.join(re.findall(pattern_gender, user.description)[0]))
        else:
          tag_gender = ''

        # タグの有無を確認
        if tag_age == '' and tag_gender == '':
          continue

        # 改行は半角スペースに置換
        tweet1 = id2status[id].full_text.replace("\n", " ")
        # スクリーンネームを正規表現を用いて削除
        tweet1 = re.sub(r"@[0-9a-zA-Z_]{1,15} +", "", tweet1)

        tweet2 = id2status[rid].full_text.replace("\n", " ")
        tweet2 = re.sub(r"@[0-9a-zA-Z_]{1,15} +", "", tweet2)
            
        f.write(tag_age + tag_gender + "\t" + tweet1 + "\t" + tweet2 + "\n")
        f1.write(user.screen_name + "\t" + tag_age + tag_gender + "\t" + tweet1 + "\t" + tweet2 + "\n")
    f.close()
    f1.close()
    print("Write " + str(len(id2replyid)) + " pairs.")
